## Programming Drill 3.2.1
### Modify your program from Programming Drill 3.1.1 so that the entries in the matrices can be fractions as opposed to Boolean values.

Changes from boolean version:

Matrix entries may now be fractions or decimals (e.g. 3/4, 0.25, 1).

The program uses fractions.Fraction for exact rational arithmetic.

Validation: each column must sum to exactly 1 (stochastic column-stochastic matrix), checked using Fraction equality.

Starting state entries can be non-negative integers or fractions (useful for probabilities or fractional marbles).

All math (matrix multiply, exponentiation) uses Fraction so results are exact rationals.

In [3]:
#!/usr/bin/env python3
"""
Marble experiment with fractional matrix entries (Programming Drill 3.2.1).

Matrix A is n x n with rational entries (Fraction). Requirement: each column must sum to 1 (column-stochastic).
State is a vector of non-negative rationals.
State evolves as: state_next = A * state
"""

from fractions import Fraction
from typing import List

def parse_fraction(s: str) -> Fraction:
    """Parse a user string into Fraction. Accepts '3/4', '0.25', '1', '-2', etc."""
    s = s.strip()
    # Allow empty -> error upstream
    try:
        # Fraction can parse strings like '3/4' and '0.25' and '1'
        return Fraction(s)
    except Exception:
        # try float fallback (should not be needed often)
        try:
            return Fraction(float(s))
        except Exception:
            raise ValueError(f"Cannot parse '{s}' as a rational number.")

def read_int(prompt: str) -> int:
    while True:
        try:
            v = int(input(prompt).strip())
            return v
        except Exception:
            print("Please enter a valid integer.")

def read_matrix(n: int) -> List[List[Fraction]]:
    print(f"Enter the matrix rows one by one. Each row should have {n} rational entries (examples: 3/4 0.25 1).")
    A: List[List[Fraction]] = []
    for i in range(n):
        while True:
            line = input(f"Row {i} : ").strip()
            parts = line.split()
            if len(parts) != n:
                print(f"Expected {n} entries; got {len(parts)}. Try again.")
                continue
            try:
                row = [parse_fraction(x) for x in parts]
            except ValueError as e:
                print(e)
                continue
            A.append(row)
            break
    return A

def validate_matrix(A: List[List[Fraction]]) -> bool:
    n = len(A)
    if any(len(row) != n for row in A):
        print("Matrix is not square.")
        return False
    # check each column sums exactly to 1
    for j in range(n):
        col_sum = sum(A[i][j] for i in range(n))
        if col_sum != Fraction(1):
            print(f"Column {j} sums to {col_sum} (require exactly 1).")
            return False
    return True

def print_matrix(A: List[List[Fraction]]):
    print("\nMatrix A:")
    for row in A:
        # print fractions as 'a/b' where appropriate
        print("  ".join(str(x) for x in row))
    print()

def read_state(n: int) -> List[Fraction]:
    while True:
        line = input(f"Enter starting state (space-separated {n} non-negative rationals, e.g. 1 0 1/2): ").strip()
        parts = line.split()
        if len(parts) != n:
            print(f"Expected {n} entries; got {len(parts)}. Try again.")
            continue
        try:
            state = [parse_fraction(x) for x in parts]
        except ValueError as e:
            print(e)
            continue
        if any(x < 0 for x in state):
            print("State entries must be non-negative. Try again.")
            continue
        return state

def mat_mult(A: List[List[Fraction]], B: List[List[Fraction]]) -> List[List[Fraction]]:
    n = len(A)
    C = [[Fraction(0) for _ in range(n)] for _ in range(n)]
    for i in range(n):
        for j in range(n):
            s = Fraction(0)
            for k in range(n):
                s += A[i][k] * B[k][j]
            C[i][j] = s
    return C

def mat_vec_mult(A: List[List[Fraction]], v: List[Fraction]) -> List[Fraction]:
    n = len(A)
    res = [Fraction(0) for _ in range(n)]
    for i in range(n):
        s = Fraction(0)
        for j in range(n):
            s += A[i][j] * v[j]
        res[i] = s
    return res

def mat_pow(A: List[List[Fraction]], exponent: int) -> List[List[Fraction]]:
    """Fast exponentiation (A^exponent) using Fraction arithmetic."""
    n = len(A)
    # identity
    result = [[Fraction(1) if i==j else Fraction(0) for j in range(n)] for i in range(n)]
    base = [row[:] for row in A]
    e = exponent
    while e > 0:
        if e & 1:
            result = mat_mult(result, base)
        base = mat_mult(base, base)
        e >>= 1
    return result

def print_state(state: List[Fraction]):
    # show fractions nicely; also show float approx in parentheses for readability
    def fmt(x: Fraction) -> str:
        if x.denominator == 1:
            return f"{x.numerator}"
        else:
            # also show approximate float for convenience
            return f"{x.numerator}/{x.denominator} ({float(x):.6g})"
    print("State:", "  ".join(fmt(x) for x in state))

def main():
    print("Marble experiment (fractional entries) — Programming Drill 3.2.1")
    n = read_int("Enter number of vertices (n): ")
    if n <= 0:
        print("n must be positive.")
        return

    A = read_matrix(n)
    print_matrix(A)

    if not validate_matrix(A):
        print("Matrix validation failed. Each column must sum to exactly 1.")
        return

    print("Matrix validated successfully.")
    print_matrix(A)

    state0 = read_state(n)
    t = read_int("Enter number of time clicks (non-negative integer): ")
    if t < 0:
        print("Number of time clicks must be non-negative.")
        return

    if t == 0:
        final_state = state0
    else:
        A_t = mat_pow(A, t)
        final_state = mat_vec_mult(A_t, state0)

    print(f"\nState after {t} time clicks:")
    print_state(final_state)

if __name__ == "__main__":
    main()



Marble experiment (fractional entries) — Programming Drill 3.2.1
Enter the matrix rows one by one. Each row should have 2 rational entries (examples: 3/4 0.25 1).

Matrix A:
1/2  0
1/2  1

Matrix validated successfully.

Matrix A:
1/2  0
1/2  1


State after 2 time clicks:
State: 1/16 (0.0625)  15/16 (0.9375)
